Original Source code from https://github.com/project-codeflare/codeflare-sdk/blob/main/demo-notebooks/guided-demos/4_gpt.ipynb

In [ ]:
# Import pieces from codeflare-sdk
from codeflare_sdk.cluster.cluster import Cluster, ClusterConfiguration
from codeflare_sdk.cluster.auth import TokenAuthentication

In [ ]:
# Create authentication object for oc user permissions
auth = TokenAuthentication(
    token = "sha256~QBcp_PWlBuBg7vy1GTQYZR-w6vRjubw488A9QvyiqsA",
    server = "https://api.mini2.mydomain.com:6443",
    skip_tls=True
)
auth.login()

In [ ]:
cluster = Cluster(ClusterConfiguration(name='gptfttest', namespace='huggingface', min_worker=2, max_worker=3,
    min_cpus=2, max_cpus=4, min_memory=8, max_memory=16, gpu=1, instascale=False))

In [ ]:
cluster.up()

In [ ]:
cluster.wait_ready()

In [ ]:
from codeflare_sdk.cluster.cluster import list_all_queued
list_all_queued(namespace="huggingface")

In [ ]:
cluster.details()

In [ ]:
from codeflare_sdk.job.jobs import DDPJobDefinition

In [ ]:
arg_list = [
    "--model_name_or_path", "gpt2",
    "--dataset_name", "wikitext",
    "--dataset_config_name", "wikitext-2-raw-v1",
    "--per_device_train_batch_size", "3", # 3 works with 16GB GPU RAM
    "--per_device_eval_batch_size", "3",
    "--do_train",
    "--do_eval",
    "--output_dir", "/tmp/test-clm",
    "--overwrite_output_dir",
    "--num_train_epochs", "3" # default is 3
]

In [ ]:
jobdef = DDPJobDefinition(
    name="gpttest",
    script="run_clm.py",
    script_args=arg_list,
    scheduler_args={"requirements": "requirements.txt"}
)
job = jobdef.submit(cluster)

In [ ]:
job.logs()

In [ ]:
job.cancel()

In [ ]:
cluster.down()

In [ ]:
auth.logout()